In [1]:
import tensorflow as tf
from tensorflow import keras 
import pandas as pd
import urllib.request
import itertools
import os
import numpy as np
import shutil

In [2]:
model = tf.keras.models.load_model('../input/land-scene-classifier/land_classifier.h5')
filenames = ["".join(p) for p in itertools.product(['0','1','2','3'], repeat=5)]
df = pd.DataFrame(filenames,columns=['path'])

In [3]:
shutil.rmtree("./images",ignore_errors=True)
os.makedirs('./images')
for i in range(len(filenames)):
    quadkey = filenames[i]
    if i%100==0:print(quadkey) 
    urllib.request.urlretrieve("http://ecn.t0.tiles.virtualearth.net/tiles/a02301023020111{}.jpeg?g=10583".format(quadkey), "./images/02301023020111{}.jpeg".format(quadkey))
print("Done.")

00000
01210
03020
10230
12100
13310
21120
22330
30200
32010
33220
Done.


In [4]:
def load_test_img(path):
    img = tf.image.decode_jpeg(tf.io.read_file("./images/02301023020111"+path+".jpeg"), channels=3)
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, [224,224])
    img = keras.applications.vgg16.preprocess_input(img)
    return img

In [5]:
eval_ds = tf.data.Dataset.from_tensor_slices((df.path.values))
eval_ds = eval_ds.map(load_test_img).batch(32)

prediction = model.predict(eval_ds)
print(prediction)

[[1.1321260e-18 4.9006507e-11 4.2318107e-11 ... 2.6155972e-06
  2.3000887e-04 5.6003927e-11]
 [5.7456554e-17 5.5318344e-11 1.9078798e-06 ... 7.9469073e-06
  4.2047221e-03 1.6762829e-10]
 [9.1822734e-16 2.3114514e-10 1.4130903e-08 ... 1.4531185e-08
  2.1764505e-01 7.7732445e-11]
 ...
 [4.3576523e-17 3.5579991e-08 3.2938117e-09 ... 8.5973821e-04
  8.8899702e-01 3.3851906e-09]
 [6.4128640e-19 1.4623790e-05 2.8156390e-05 ... 4.4065536e-04
  7.1828380e-02 3.8767348e-07]
 [1.8157522e-18 4.7155894e-11 1.6413388e-09 ... 3.6693893e-07
  2.9152203e-01 6.7174849e-11]]


In [6]:
ans = []
for i in range(len(prediction)):
    ans.append(np.argmax(prediction[i]))
df = pd.concat([df,pd.DataFrame(ans,columns=['ans'])],axis=1)
df.to_csv('map_data.csv')
df.head()

,path,ans
0,00000,11
1,00001,17
2,00002,11
3,00003,19
4,00010,11
